### Import libraries and set up Redshift connnection string

In [1]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import os

In [2]:
redshift_endpoint = os.getenv("REDSHIFT_ENDPOINT")
redshift_user = os.getenv("REDSHIFT_USER")
redshift_pass = os.getenv("REDSHIFT_PASS")
port = 5439
dbname = 'prod'

engine_str = "redshift+psycopg2://%s:%s@%s:%s/%s" % (redshift_user,redshift_pass,redshift_endpoint,str(port),dbname)

engine = create_engine(engine_str)

### Pull sample data from Redshift into a Pandas dataframe
show the first 5 rows of the data

In [3]:
mx_data = pd.read_sql_query('select * from edw.mhx_claims limit 100;', engine)
mx_data.head()

,claim_txn_id,srvc_line_seq,payer_plan_type,procedure_cd,procedure_dt,procedure_mod1_cd,procedure_mod2_cd,procedure_type_cd,patient_id,place_of_srvc,...,diag7_cd,diag8_cd,diag9_cd,billing_prov_id,service_units,claim_proc_dt,revenue_cd,performing_prov_npi,billing_npi,srvc_year
0,5.441200e+14,2.0,Third Party,99213,2011-02-10,None,None,MX,147475816.0,11,...,None,None,None,000000000,1.0,2011-02-25,None,043760551,000000000,2011
1,5.667400e+14,3.0,Third Party,J3301,2011-03-15,None,None,MX,141714062.0,11,...,None,None,None,000000000,1.0,2011-04-16,None,050553206,000000000,2011
2,6.445600e+14,3.0,Third Party,90716,2011-08-15,None,None,MX,189438190.0,11,...,None,None,None,000000000,1.0,2011-08-23,None,030417049,000000000,2011
3,6.445600e+14,4.0,Third Party,90460,2011-08-15,None,None,MX,189438190.0,11,...,None,None,None,000000000,1.0,2011-08-23,None,030417049,000000000,2011
4,6.749200e+14,1.0,Third Party,95811,2011-07-29,26,None,MX,116624639.0,22,...,None,None,None,000000000,1.0,2011-09-10,None,020550647,000000000,2011


### Group data by payer_plan_type with counts of rows

In [4]:
mx_data.groupby(['payer_plan_type']).count()

,claim_txn_id,srvc_line_seq,procedure_cd,procedure_dt,procedure_mod1_cd,procedure_mod2_cd,procedure_type_cd,patient_id,place_of_srvc,srvc_from_dt,...,diag7_cd,diag8_cd,diag9_cd,billing_prov_id,service_units,claim_proc_dt,revenue_cd,performing_prov_npi,billing_npi,srvc_year
payer_plan_type,,,,,,,,,,,,,,,,,,,,,
Medicaid,5,5,5,5,0,0,5,5,5,5,...,0,0,0,5,5,5,0,5,5,5
Medicare,1,1,1,1,0,0,1,1,1,1,...,0,0,0,1,1,1,0,1,1,1
Third Party,94,94,94,94,11,0,94,94,94,94,...,0,0,0,94,94,94,0,94,94,94


### Group data by payer_plan_type with counts of rows on 1 column

In [5]:
mx_data[['payer_plan_type', 'claim_txn_id']].groupby(['payer_plan_type']).count()

,claim_txn_id
payer_plan_type,
Medicaid,5
Medicare,1
Third Party,94


### Import plotting library

In [6]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls


In [7]:
mx_ppt = mx_data[['payer_plan_type', 'claim_txn_id']].groupby(['payer_plan_type']).count()
mx_ppt
mx_ppt.columns

layout = go.Layout(title="Payer Plan Type", yaxis=dict(title="Count"))
data = [go.Bar(x=mx_ppt.index, y=mx_ppt.claim_txn_id)]
py.iplot(go.Figure(data=data, layout=layout))

